In [36]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

#TODO: subtract outgoing

import urllib.request, json
import csv
import itertools
import math
import collections
import sys

import grequests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import utils


class fts(object):
    def __init__(self, sc, test=None, all_sector=False):
        self.all_sector = all_sector
        self.test = test
        self.sc = sc

        self.fts = self.master_pull()
        self.merge_sc()
        self.flows = self.extract_flows()

        # remove flows col as it's quite large
        self.fts = self.fts.drop('fts.data.flows', axis=1)

    def extract_flows(self):
        """take all flows and add to df"""

        flow_df = pd.DataFrame()
        for v in self.fts.iterrows():
            norm = json_normalize(v[1]['fts.data.flows'])
            norm['fts.uid'] = v[1]['sc.uid']

            if len(flow_df) == 0:
                flow_df = norm
            else:
                # need to make sure we're adding t3h same c0lz
                assert (norm.columns.all(flow_df.columns))
                flow_df = flow_df.append(norm, ignore_index=True)

        return flow_df

    def master_pull(self):
        """group by year, country and pull down total values with optional filter by cluster.
            returns: dict of {sector 1 : info df}
        """
        countries = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/location")
                                              .read().decode()))['data']

        print('pulled countries')

        self.hist = self.sc

        def get_other_sects():
            """get comma seperated string of all other sectors if needed"""
            r = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/global-cluster")
                                          .read().decode()))['data']

            r = [v['code'] for v in r[0]]
            return ','.join(r)

        #set equal just SHL, and if we're looking for others then get 'em
        clusters = ['SHL']
        cluster_vals = {}

        if self.all_sector:
            cluster = get_other_sects()

        self.hist['sc.fts_url'] = self.hist.apply(lambda x: \
                          'https://api.hpc.tools/v1/public/fts/flow?countryISO3={0}&year={1}' \
                          '&globalClusterCode={2}'.format(x['sc.iso3'], x['sc.year'], \
                                                          'SHL,WSH,FSC,CSS,EDU,ERY,HEA' if not self.all_sector else get_other_sects()), axis=1)

        if self.test:
            hrefz = self.hist['sc.fts_url'][:self.test]
        else:
            hrefz = self.hist['sc.fts_url']

        def exception_handler(request, exception):
            print('Bad URL for ' + str(request))

        # re.match('.*globalClusterCode=(.*)', good_resps[0]['url']).groups()[0]
        resps = []
        rs = (grequests.get(ref) for ref in hrefz)
        resps += grequests.map(rs, exception_handler=exception_handler, size=25)

        good_resps = []
        bad_resps = []
        for r in resps:
            load = json.loads(r.content)
            load['url'] = r.url
            if r.status_code == 200:
                good_resps.append(load)
            else:
                bad_resps.append(load)

        print('num bad resps: ' + str(len(bad_resps)))

        resps = utils.api_pull(hrefz, False)

        return json_normalize(good_resps).add_prefix('fts.')

    def merge_sc(self):
        self.fts = self.fts.merge(self.hist, left_on='fts.url', right_on='sc.fts_url')

    def get_flow_bdown(self):
        """break down flows by needed columns"""

        def hp(v):
            if 'Plan' in json_normalize(v['destinationObjects']).values or 'Plan' in json_normalize(
                    v['sourceObjects']).values:
                return 'has_plan'
            else:
                return 'no_plan'

        self.flows['plan'] = self.flows.apply(lambda x: hp(x), axis=1)

        # we are skipping boundary for now as it's <<<<
        d = self.flows[['fts.uid', 'plan', 'amountUSD']].groupby(['fts.uid', 'plan'], as_index=False).aggregate(
            {'amountUSD': 'sum'})
        piv = pd.pivot_table(d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[d['plan']] \
                             , fill_value=0).reset_index()
        piv['total'] = piv['has_plan'] + piv['no_plan']
        return piv

#
# import sc_pull
# sc = sc_pull.pull()
# # s_f = fts(test=10, sc=sc)
# all_f = fts(test=10, sc=sc, all_sector=True)
#
# # print(s_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 sc
# print(all_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 all
#
# # AGR,CCM,PRO-CPN,CSS,ERY,EDU,TEL,FSC,PRO-GBV,HEA,PRO-HLP,LOG,PRO-MIN,MS,NUT,OTH,PRO,WSH


In [37]:
import sc_pull
sc = sc_pull.pull()
s_f = fts(test=2, sc=sc)
# all_f = fts(test=4, sc=sc, all_sector=True)

179,903,300
197,823,342

loaded 227 SC entries
pulled countries
num bad resps: 0
pulling for : https://api.hpc.tools/v1/public/fts/flow?countryISO3=PAK&year=2005&globalClusterCode=SHL,WSH,FSC,CSS,EDU,ERY,HEA
pulled. num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


(197, 823, 342)

In [38]:
all_f.flows['fts.uid'].value_counts()

s_f.flows['fts.uid'].value_counts()

PAK2005    200
COD2006    200
Name: fts.uid, dtype: int64

In [26]:
all_f.flows

,amountUSD,boundary,budgetYear,contributionType,createdAt,date,decisionDate,description,destinationObjects,exchangeRate,...,onBoundary,originalAmount,originalCurrency,parentFlowId,refCode,reportDetails,sourceObjects,status,updatedAt,versionId
0,195340,incoming,2005,in kind,2016-04-12T16:32:00Z,2005-11-12T00:00:00Z,2005-11-12T00:00:00Z,"PAKISTAN - Eathquake - November 2005, blanket ...","[{'type': 'Organization', 'id': '83', 'name': ...",2150.097,...,single,420000000.0,BYR,NaN,NaN,"[{'sourceType': 'Primary', 'organization': 'Be...","[{'type': 'Organization', 'id': '4837', 'name'...",commitment,2016-04-12T16:32:00Z,1
1,160000,incoming,2000,in kind,2005-02-14T00:00:00Z,2005-02-13T00:00:00Z,2005-02-13T00:00:00Z,"Emergency relief items (food, water filter clo...","[{'type': 'Organization', 'id': '2915', 'name'...",NaN,...,single,NaN,USD,NaN,NaN,NaN,"[{'type': 'Organization', 'id': '2915', 'name'...",paid,2005-03-04T00:00:00Z,1
2,20000,incoming,NaN,in kind,2005-02-14T00:00:00Z,2005-02-13T00:00:00Z,2005-02-13T00:00:00Z,Tents and other shelter items*,"[{'type': 'Organization', 'id': '4397', 'name'...",NaN,...,single,0.0,NaN,NaN,NaN,NaN,"[{'type': 'Organization', 'id': '4397', 'name'...",paid,2005-02-14T00:00:00Z,1
3,25000,incoming,NaN,in kind,2005-02-14T00:00:00Z,2005-02-13T00:00:00Z,2005-02-13T00:00:00Z,Emergency medical supplies,"[{'type': 'Organization', 'id': '4398', 'name'...",NaN,...,single,NaN,USD,NaN,NaN,NaN,"[{'type': 'Organization', 'id': '4398', 'name'...",paid,2005-02-14T00:00:00Z,1
4,50000,incoming,2000,financial,2005-02-14T00:00:00Z,2005-02-13T00:00:00Z,2005-02-13T00:00:00Z,Emergency cash grant for the purchase of tents...,"[{'type': 'Organization', 'id': '5294', 'name'...",NaN,...,single,NaN,USD,NaN,NaN,"[{'sourceType': 'Primary', 'organization': 'Of...","[{'type': 'Organization', 'id': '4396', 'name'...",paid,2005-02-16T00:00:00Z,1
5,25000,incoming,NaN,financial,2005-02-16T00:00:00Z,2005-02-15T00:00:00Z,2005-02-15T00:00:00Z,To support educational facilities,"[{'type': 'Organization', 'id': '2993', 'name'...",NaN,...,single,NaN,USD,NaN,NaN,"[{'sourceType': 'Primary', 'organization': 'Un...","[{'type': 'Organization', 'id': '2993', 'name'...",paid,2005-02-16T00:00:00Z,1
6,100000,incoming,NaN,financial,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,Emergency relief assistance,"[{'type': 'Organization', 'id': '4305', 'name'...",NaN,...,single,NaN,USD,NaN,NaN,"[{'sourceType': 'Primary', 'organization': Non...","[{'type': 'Organization', 'id': '2933', 'name'...",paid,2005-02-17T00:00:00Z,1
7,200000,incoming,NaN,in kind,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,In-kind donation,"[{'type': 'Organization', 'id': '83', 'name': ...",NaN,...,single,0.0,NaN,NaN,NaN,"[{'sourceType': 'Primary', 'organization': Non...","[{'type': 'Organization', 'id': '100', 'name':...",paid,2005-02-17T00:00:00Z,1
8,42372,incoming,NaN,financial,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,"Cash grant - CHF 50,000","[{'type': 'Organization', 'id': '8538', 'name'...",1.180,...,single,50000.0,CHF,NaN,NaN,"[{'sourceType': 'Primary', 'organization': Non...","[{'type': 'Organization', 'id': '4393', 'name'...",paid,2005-02-17T00:00:00Z,1
9,100000,incoming,NaN,financial,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,2005-02-17T00:00:00Z,Cash grant,"[{'type': 'Organization', 'id': '83', 'name': ...",NaN,...,single,NaN,USD,NaN,NaN,"[{'sourceType': 'Primary', 'organization': Non...","[{'type': 'Organization', 'id': '2976', 'name'...",paid,2005-02-17T00:00:00Z,1
